In [1]:
# assign directory
import git
from pathlib import Path
import os
import numpy as np
import pywt
import matplotlib.pyplot as plt
import nibabel as nib
import random
from scipy import stats
from scipy import fft
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle
from tqdm import tqdm
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
from plotting import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))

# data_dir = '/Users/hannahchung/Downloads/cleaned/'
# file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename != ".DS_Store"]
# file_names = os.listdir(data_dir)
import joblib

#Reassigning directory to cleaned + jittered 
data_dir = '/Users/hannahchung/Downloads/cleaned_jittered/'
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename != ".DS_Store"]
file_names = os.listdir(data_dir)

/Users/hannahchung/Documents/GitHub/hierarchical-bayesian-model-validation/utilities/testing.py:14: UserWarning: legacy printing option can currently only be '1.13', '1.21', or `False`
  np.set_printoptions(legacy='1.25')


In [ ]:
# orientation_direction = {
#     'aaa': 'Smooth Volume',
#     'aad': 'Z Detail',
#     'ada': 'Y Detail',
#     'add': 'Y-Z Diagonal',
#     'daa': 'X Detail',
#     'dad': 'X-Z Diagonal',
#     'dda': 'X-Y Diagonal',
#     'ddd': 'XYZ Diagonal',
#     'L1': 'Approximation Layer'
# }

# axis_description = {
#     'aaa': 'Smooth overall brain structure',
#     'aad': 'Brainstem → Crown',
#     'ada': 'Face → Back of head',
#     'add': 'Front-bottom → Back-top',
#     'daa': 'Right → Left hemisphere',
#     'dad': 'Right-bottom → Left-top',
#     'dda': 'Right-front → Left-back',
#     'ddd': 'All-direction fine features',
#     'L1': 'Low-freq base structure'
# }

# gray_df["First_label"] = gray_df["First"].map(axis_description)
# gray_df["Second_label"] = gray_df["Second"].map(axis_description)


In [ ]:
# Double-check dtype of 3d mri: float32
# file_path = os.path.join(data_dir, file_names[0])
# data = np.load(file_path)
# image = data['arr_0']

# print("Image dtype:", image.dtype)

# voxel = image[100, 100, 100]
# print("Single voxel value:", voxel)
# print("Single voxel dtype:", type(voxel))

Image dtype: float32
Single voxel value: 94.15749
Single voxel dtype: <class 'numpy.float32'>


Jitter function

In [3]:
def npz_opener_with_jitter(path):
    arr = np.load(path)["arr_0"].astype(np.float32)
    jitter = np.random.uniform(-0.5, 0.5, arr.shape).astype(np.float32)
    arr = np.where(np.isnan(arr), np.nan, arr + jitter)
    return arr

Add jitter + save files

In [ ]:
# src_dir    = '/Users/hannahchung/Downloads/cleaned/'
# jitter_dir = '/Users/hannahchung/Downloads/cleaned_jittered/'
# os.makedirs(jitter_dir, exist_ok=True)

# for fname in tqdm(os.listdir(src_dir)):
#     if not fname.endswith('.npz'):
#         continue
#     arr = npz_opener_with_jitter(os.path.join(src_dir, fname))
#     np.savez_compressed(os.path.join(jitter_dir, fname), arr_0=arr)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [2]:
def convert_to_wavelet_basis_3d(folder_dir, basis="db1", normalized = False):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir) if filename.endswith('.npz')]
    #Setup df Dict
    data = np.load(file_list[0])
    image = data['arr_0']
    first_image = pywt.wavedecn(image, basis)
    layer_len = len(first_image)
    direction_names = first_image[1].keys()
    direction_num = len(direction_names)
    print(str(layer_len) + " layers being used")
    

    #Fill DF DICT
    layer_arr = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    orientation = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    data_arr = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    cnt = 0
    for k in tqdm(range(len(file_list))):

        data = np.load(file_list[k])
        image = data['arr_0']


        if normalized:
            std= np.std(image)
            mean = np.mean(image)
            image = (image- mean)/std

    
        transformed = pywt.wavedecn(image, 'db1')
        

        arr = transformed[0].flatten()
        layer_arr[cnt] = 1
        orientation[cnt] =  "L1"
        data_arr[cnt] = arr.flatten()
        cnt += 1

        for i in range(1, layer_len): 
            for j in direction_names:
                
                arr = np.array(transformed[i][j]).flatten()
                layer_arr[cnt] = i+1
                orientation[cnt] =  j
                data_arr[cnt] = arr.flatten()
                cnt += 1

    df = pd.DataFrame()

    df["layer"] = layer_arr
    df["orientation"] = orientation
    df["data"] = data_arr
    new_df = pd.DataFrame(columns=["layer", "orientation", "data"])
    for lo, sf in df.groupby(["layer", "orientation"])[["data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
        new_df.loc[len(new_df.index)] = [lo[0], lo[1],  np.hstack(sf['data'])]
    
    return new_df

In [7]:
def ksCombinations(samps, samp_names, layer):
    df = pd.DataFrame(columns = ["Layer", "First", "Second", "KS_Stat", "Pvalue"])
    for i in tqdm(range(len(samps))):
        for j in range(i+1, len(samps)):
            s1 = samps[i][np.isfinite(samps[i])]
            s2 = samps[j][np.isfinite(samps[j])]

            if len(s1) == 0 or len(s2) == 0:
                continue 
            ksres = stats.ks_2samp(s1, s2)
            df.loc[len(df)] = [layer, samp_names[i], samp_names[j], ksres.statistic, ksres.pvalue]
    return df
            

In [4]:
expected_dir = '/Users/hannahchung/Downloads/cleaned_jittered/'
import pathlib
print("data_dir is set to:", data_dir)
print("expected_dir     :", expected_dir)
print()
if pathlib.Path(data_dir).resolve() == pathlib.Path(expected_dir).resolve():
    print("data_dir points to cleaned_jittered")
else:
    print("data_dir is NOT the jittered folder!")

data_dir is set to: /Users/hannahchung/Downloads/cleaned_jittered/
expected_dir     : /Users/hannahchung/Downloads/cleaned_jittered/

data_dir points to cleaned_jittered


# Gray

In [5]:
gray_wv = convert_to_wavelet_basis_3d(data_dir)
gray_wv.head()


8 layers being used


  0%|          | 0/1000 [00:00<?, ?it/s]

,layer,orientation,data
0,1,L1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,2,aad,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,2,ada,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,2,add,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,2,daa,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [8]:
jittered_gray_df = pd.DataFrame(columns = ["Layer", "First", "Second", "KS_Stat", "Pvalue"])
for i in range(2, 11):
    layer_df = gray_wv[gray_wv["layer"] == i]
    data = layer_df["data"].to_list()
    names = layer_df["orientation"].to_list()
    ks_df = ksCombinations(data, layer_df["orientation"].to_list(), i).sort_values(["Layer", "KS_Stat"])
    jittered_gray_df = pd.concat([jittered_gray_df, ks_df], axis = 0)
jittered_gray_df

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

/var/folders/cg/nwm9l18x22zc8_fj15gvgg2h0000gp/T/ipykernel_44464/1923987592.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jittered_gray_df = pd.concat([jittered_gray_df, ks_df], axis = 0)


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

/var/folders/cg/nwm9l18x22zc8_fj15gvgg2h0000gp/T/ipykernel_44464/1923987592.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jittered_gray_df = pd.concat([jittered_gray_df, ks_df], axis = 0)


0it [00:00, ?it/s]

/var/folders/cg/nwm9l18x22zc8_fj15gvgg2h0000gp/T/ipykernel_44464/1923987592.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jittered_gray_df = pd.concat([jittered_gray_df, ks_df], axis = 0)


,Layer,First,Second,KS_Stat,Pvalue
11,3,add,daa,0.081918,2.409579e-03
15,3,daa,dad,0.081918,2.409579e-03
20,3,dda,ddd,0.119880,1.100344e-06
12,3,add,dad,0.128871,1.160573e-07
19,3,dad,ddd,0.212787,2.992032e-20
...,...,...,...,...,...
1,8,aad,add,0.189767,0.000000e+00
3,8,aad,dad,0.196239,0.000000e+00
10,8,ada,ddd,0.259576,0.000000e+00
17,8,daa,ddd,0.261190,0.000000e+00


In [9]:
import joblib
joblib.dump(jittered_gray_df, 'jittered_gray_df.pkl')
# gray_df = joblib.load('gray_df.pkl')


['jittered_gray_df.pkl']

In [11]:
jittered_gray_df = joblib.load('jittered_gray_df.pkl')

In [12]:
jittered_gray_df

,Layer,First,Second,KS_Stat,Pvalue
11,3,add,daa,0.081918,2.409579e-03
15,3,daa,dad,0.081918,2.409579e-03
20,3,dda,ddd,0.119880,1.100344e-06
12,3,add,dad,0.128871,1.160573e-07
19,3,dad,ddd,0.212787,2.992032e-20
...,...,...,...,...,...
1,8,aad,add,0.189767,0.000000e+00
3,8,aad,dad,0.196239,0.000000e+00
10,8,ada,ddd,0.259576,0.000000e+00
17,8,daa,ddd,0.261190,0.000000e+00


In [ ]:
def multiSampleComparisonPlots(data, samp_names, bw=0.2, hist_plot=True):
    fig, axes = plt.subplots(1, 3, figsize=(24, 6))
    for i in range(len(data)):
        samp = data[i]
        name = samp_names[i]
        n = len(samp)

        sns.kdeplot(ax=axes[0], x=samp, bw_method=bw, label=name)
        sns.kdeplot(ax=axes[1], x=samp, bw_method=bw, log_scale=[False, True], label=name)
        axes[2].plot(np.sort(samp), np.arange(1, n+1)/n, label=name)

        if hist_plot:
            sns.histplot(ax=axes[0], x=samp, stat='density', label=name, alpha=0.2)
            sns.histplot(ax=axes[1], x=samp, stat='density', log=True, label=name, alpha=0.2)

    axes[0].set_title("Non-Log PDF")
    axes[1].set_title("Log-Scale PDF")
    axes[2].set_title("CDF")
    for ax in axes: ax.legend()
    return fig


In [29]:
def twoSampleComparisonPlots(gray_wv, layer_num, ori1, ori2, bw=0.2, alpha=0.2, plot_hist=False):
    df = gray_wv[gray_wv["layer"] == layer_num]
    samp1_row = df[df["orientation"] == ori1]
    samp2_row = df[df["orientation"] == ori2]

    if samp1_row.empty or samp2_row.empty:
        print(f"One or both orientations not found in layer {layer_num}: {ori1}, {ori2}")
        return

    s1 = samp1_row["data"].values[0]
    s2 = samp2_row["data"].values[0]
    name1 = samp1_row["axis_description"].values[0]
    name2 = samp2_row["axis_description"].values[0]

    ksres = stats.ks_2samp(s1, s2)
    ks_stat = ksres.statistic
    ks_pval = ksres.pvalue

    fig, axes = plt.subplots(1, 3, figsize=(24, 6))
    sns.kdeplot(ax=axes[0], x=s1, bw_method=bw, label=name1)
    sns.kdeplot(ax=axes[0], x=s2, bw_method=bw, label=name2)
    sns.kdeplot(ax=axes[1], x=s1, bw_method=bw, log_scale=[False, True], label=name1)
    sns.kdeplot(ax=axes[1], x=s2, bw_method=bw, log_scale=[False, True], label=name2)

    if plot_hist:
        sns.histplot(ax=axes[0], x=s1, stat='density', alpha=alpha, label=name1)
        sns.histplot(ax=axes[0], x=s2, stat='density', alpha=alpha, label=name2)
        sns.histplot(ax=axes[1], x=s1, stat='density', log=True, alpha=alpha, label=name1)
        sns.histplot(ax=axes[1], x=s2, stat='density', log=True, alpha=alpha, label=name2)

    axes[2].plot(np.sort(s1), np.arange(1, len(s1)+1)/len(s1), label=name1)
    axes[2].plot(np.sort(s2), np.arange(1, len(s2)+1)/len(s2), label=name2)

    axes[0].set_title("Non-Log PDF")
    axes[1].set_title("Log-Scale PDF")
    axes[2].set_title(f"CDF (p={ks_pval:.2e}, D={ks_stat:.4f})")
    for ax in axes: ax.legend()
    return fig
